## Comparision of Summaries FROM OPENAI VS GEMINI

In [1]:
from langchain.document_loaders import YoutubeLoader
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
import google_auth_oauthlib.flow
import googleapiclient.discovery
from google.auth.transport.requests import Request
from dotenv import load_dotenv
import os
import time  # Import the time module

# Load environment variables
load_dotenv()


True

In [2]:
# Replace with your own channel names or URLs
channels_to_follow = ["mkbhd", "freecodecamp", "techwithtim"]


In [3]:
# Set up YouTube API credentials
api_key = os.getenv("YOUTUBE_API_KEY")
API_KEY = api_key
API_SERVICE_NAME = "youtube"
API_VERSION = "v3"


In [4]:
# Function to get the latest videos from specified channels
def get_latest_videos(api_key, channels):
    youtube = googleapiclient.discovery.build(API_SERVICE_NAME, API_VERSION, developerKey=api_key)

    videos = []
    for channel in channels:
        # Get channel ID
        search_response = youtube.search().list(q=channel, type="channel", part="id").execute()
        channel_id = search_response["items"][0]["id"]["channelId"]

        # Get latest videos from the channel
        playlist_response = youtube.channels().list(id=channel_id, part="contentDetails").execute()
        playlist_id = playlist_response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

        videos_response = youtube.playlistItems().list(playlistId=playlist_id, part="snippet", maxResults=5).execute()
        
        # Add channel information to each video
        for video in videos_response["items"]:
            video["channel"] = channel
            videos.append(video)

    return videos

In [5]:
# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", api_key=api_key)

# Get videos from the channels (assuming latest_videos is available)
latest_videos = get_latest_videos(API_KEY, channels_to_follow)

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

# Load and summarize videos
class TextWrapper:
    def __init__(self, content, metadata=None):
        self.page_content = content
        self.metadata = metadata or {}

texts = []

WARNING! api_key is not default parameter.
                    api_key was transferred to model_kwargs.
                    Please confirm that api_key is what you intended.


## Testing with OpenAI GPT Model using langchain

In [6]:
# for every 30 seconds, it will retrieve one video and then summarize it
for video in latest_videos:
    try:
        # Get video URL, title, and channel information
        video_url = f"https://www.youtube.com/watch?v={video['snippet']['resourceId']['videoId']}"
        video_title = video['snippet']['title']
        channel_title = video['snippet']['channelTitle']

        # Load video content
        loader = YoutubeLoader.from_youtube_url(video_url, add_video_info=True)
        result = loader.load()

        # Include video title and channel information in the summarization
        text_with_info = f"Video Title: {video_title}\nChannel: {channel_title}\n\n{result}"
        
        # Wrap the text in a simple class to mimic expected structure
        texts.append(TextWrapper(text_with_info, metadata={"video_url": video_url, "video_title": video_title}))
        
    except ValueError as e:
        print(f"Error processing video with URL {video_url}: {e}") 

In [7]:
# Load and run summarization chain if texts list is not empty
if texts:
    # Load and run summarization chain
    print("Texts before summarization:", texts)

    chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)

    for text in texts:
        # Summarize the video content
        chain.run([text])

        # Introduce a delay of 30 seconds before summarizing the next video
        time.sleep(30)

Texts before summarization: [<__main__.TextWrapper object at 0x12470dc10>, <__main__.TextWrapper object at 0x106bf9f10>, <__main__.TextWrapper object at 0x12470da60>, <__main__.TextWrapper object at 0x102f9d2e0>, <__main__.TextWrapper object at 0x124497dc0>, <__main__.TextWrapper object at 0x124497e80>, <__main__.TextWrapper object at 0x12475e070>, <__main__.TextWrapper object at 0x12475efa0>, <__main__.TextWrapper object at 0x12475ed90>, <__main__.TextWrapper object at 0x12475eee0>, <__main__.TextWrapper object at 0x1062ea370>, <__main__.TextWrapper object at 0x124488f70>, <__main__.TextWrapper object at 0x1243526a0>, <__main__.TextWrapper object at 0x107039e50>, <__main__.TextWrapper object at 0x12475e3a0>]


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Video Title: Nothing Phone 2A Review: Flashy Good Deal!
Channel: Marques Brownlee

[Document(page_content='(cube bounces) (ele

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-9S6BvFHlZO0c19LM2TkWffly on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Marques Brownlee discusses the advancements in AI-generated videos, specifically focusing on OpenAI's new model called Sora. He showcases examples of AI-generated videos that are impressively realistic, but also points out flaws. Brownlee highlights the potential impact of this technology on stock footage, video licensing, and the need for safety measures to prevent misuse, especially during an election year. He concludes by pondering the future implications of AI creativity and innovation."


CONCISE SUMMARY:

> Finished chain.

> Finished chain.


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Video Title: Top 10 Javascript Algorithms to Prepare for Coding Interviews
Channel: freeCodeCamp.org

[Document(page_content="this course will prepare you for elite co

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-9S6BvFHlZO0c19LM2TkWffly on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"The video titled "Top 10 Javascript Algorithms to Prepare for Coding Interviews" by freeCodeCamp.org covers important JavaScript algorithms for coding interviews. The video includes explanations and demonstrations of various algorithms, such as reversing a string, reversing an integer, checking for palindromes, and solving the FizzBuzz challenge. The video also provides tips and solutions for each algorithm, making it a valuable resource for preparing for coding interviews."


CONCISE SUMMARY:

> Finished chain.

> Finished chain.


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Video Title: Back End Developer Roadmap 2024
Channel: freeCodeCamp.org

[Document(page_content="in this video I'm going to explain all the core Technologies you need to learn to become

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-9S6BvFHlZO0c19LM2TkWffly on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"The video titled "Back End Developer Roadmap 2024" by freeCodeCamp.org provides a comprehensive overview of the core technologies and skills needed to become a backend developer. The roadmap covers topics such as server-side logic, database management, API development, server management, security, and more. It emphasizes the importance of continuous learning, staying updated with technological trends, and engaging with the developer community. The video also highlights essential technologies such as programming languages, version control systems, relational databases, APIs, caching, security, testing, software design, message brokers, containerization technologies, web servers, and more. The roadmap serves as a starting point for aspiring backend developers to build powerful, efficient, and secure backend systems."


CONCISE SUMMARY:

> Finished chain.

> Finished

InvalidRequestError: This model's maximum context length is 16385 tokens. However, your messages resulted in 18863 tokens. Please reduce the length of the messages.

#### API Limit Done 

## Testing With Gemini

In [6]:
import google.generativeai as genai
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled
import googleapiclient.discovery


In [7]:
# Configure Google API
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

API_SERVICE_NAME = "youtube"
API_VERSION = "v3"


In [8]:
prompt = """You are YouTube video summarizer. You will be taking the transcript text
and summarizing the entire video, providing the important summary in points
within 250 words. Please provide the summary of the text given here:  """

In [9]:
# Function to get the latest videos from specified channels
def get_latest_videos(api_key, channels):
    youtube = googleapiclient.discovery.build(API_SERVICE_NAME, API_VERSION, developerKey=api_key)

    videos = []
    for channel in channels:
        # Get channel ID
        search_response = youtube.search().list(q=channel, type="channel", part="id").execute()
        channel_id = search_response["items"][0]["id"]["channelId"]

        # Get latest videos from the channel
        playlist_response = youtube.channels().list(id=channel_id, part="contentDetails").execute()
        playlist_id = playlist_response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

        videos_response = youtube.playlistItems().list(playlistId=playlist_id, part="snippet", maxResults=5).execute()
        
        # Add channel information to each video
        for video in videos_response["items"]:
            video["channel"] = channel
            videos.append(video)

    return videos

In [10]:
# Function to extract transcript details and generate summaries
def extract_transcript_details(youtube_video_url):
    try:
        video_id = youtube_video_url.split("=")[1]
        transcript_text = YouTubeTranscriptApi.get_transcript(video_id)

        transcript = ""
        for i in transcript_text:
            transcript += " " + i["text"]

        return transcript

    except TranscriptsDisabled:
        print("Transcripts are disabled for this video. Please enable subtitles on the video.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [11]:
# Function to generate summary
def generate_gemini_content(transcript_text, prompt):
    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(prompt + transcript_text)
    return response.text


In [12]:
# Function to process each video
def process_video(video):
    try:
        # Get video URL, title, and channel information
        video_url = f"https://www.youtube.com/watch?v={video['snippet']['resourceId']['videoId']}"
        video_title = video['snippet']['title']
        channel_title = video['channel']

        # Load video content
        transcript_text = extract_transcript_details(video_url)

        # Include video title and channel information in the summarization
        text_with_info = f"Video Title: {video_title}\nChannel: {channel_title}\n\n{transcript_text}"

        # Generate summary
        summary = generate_gemini_content(transcript_text, prompt)

        print(f"Title: {video_title}\nChannel: {channel_title}\nSummary: {summary}\n---")

    except ValueError as e:
        print(f"Error processing video with URL {video_url}: {e}")


In [13]:
# Example usage
if __name__ == "__main__":
    channels_to_follow = ["mkbhd", "freecodecamp", "techwithtim"]
    latest_videos = get_latest_videos(API_KEY, channels_to_follow)

    # Process each video
    for video in latest_videos:
        process_video(video)

Title: Nothing Phone 2A Review: Flashy Good Deal!
Channel: mkbhd
Summary: **Summary:**

* Budget phones often involve trade-offs, which vary depending on the model.
* The Nothing Phone 2A stands out for its unique design, featuring a transparent back, asymmetrical layout, and glyph interface.
* Despite its low price, the 2A boasts a giant 6.7" AMOLED display with a variable refresh rate of up to 120Hz.
* Its battery life is impressive with a 5000mAh battery and 45-watt wire charging.
* Performance is respectable, thanks to the efficient MediaTek Dimensity 7200 Pro chip and Nothing OS 2.5 with its smooth and fast experience.
* The camera is serviceable, with a 50-megapixel primary lens and an ultra-wide camera.
* Notable features include an optical fingerprint reader, built-in Nothing widgets, third-party icon pack support, and glyph lighting with limited customization.
* Despite being a budget-friendly option, the 2A retains many features from Nothing's flagship models, providing a com

InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting

### Same case here API limit is done ... i'll try with local models next